# Анализ данных о венчурных фондах и инвестициях в компании-стартапы

### Описание проекта

Работа с базой данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы. Эта база данных основана на датасете Startup Investments, опубликованном на популярной платформе для соревнований по исследованию данных Kaggle. 

**Задача**: проанализировать данные о фондах и инвестициях и написать запросы к базе с помощью SQL.

#### Описание данных

- `acquisition` — cодержит информацию о покупках одних компаний другими.
- `company` — cодержит информацию о компаниях-стартапах.
- `education` — хранит информацию об уровне образования сотрудников компаний.
- `fund` — хранит информацию о венчурных фондах. 
- `funding_round` — содержит информацию о раундах инвестиций. 
- `investment` — содержит информацию об инвестициях венчурных фондов в компании-стартапы.
- `people` — содержит информацию о сотрудниках компаний-стартапов.

### Запросы

1. Все записи из таблицы `company по компаниям`, которые закрылись.

In [ ]:
SELECT *
FROM company
WHERE status = 'closed';

2. Количество привлечённых средств для новостных компаний США из таблицы company. Сортировка по убыванию значений в поле funding_total.

In [ ]:
SELECT funding_total
FROM company
WHERE country_code = 'USA'
    AND category_code = 'news'
ORDER BY funding_total DESC;

3. Общая сумму сделок по покупке одних компаний другими в долларах. Сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' 
    AND CAST(acquired_at AS date) BETWEEN '2011-01-01' AND '2013-12-31';

4. Имя, фамилия и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT
    first_name,
    last_name,
    twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

5. Информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT * 
FROM people
WHERE twitter_username LIKE '%money%'
    AND last_name LIKE 'K%';

6.  Общая сумма привлечённых инвестиций, для каждой страны, которую получили компании, зарегистрированные в этой стране. Сортировка данных по убыванию суммы.

In [ ]:
SELECT country_code, SUM(funding_total) 
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

7. Составление таблицы, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. В  итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT 
    CAST(funded_at AS date),
    MAX(raised_amount),
    MIN(raised_amount)
FROM funding_round 
GROUP BY funded_at
HAVING MIN(raised_amount) !=0
    AND MIN(raised_amount) != MAX(raised_amount);

8. Создание поля с категориями:
- Для фондов, которые инвестируют в 100 и более компаний - категория high_activity.
- Для фондов, которые инвестируют в 20 и более компаний до 100 - категория middle_activity.
- Если количество инвестируемых компаний фонда не достигает 20 - категория low_activity.

Отображены все поля таблицы fund и новое поле с категориями.


In [ ]:
SELECT *,
CASE
    WHEN invested_companies >= 100 THEN 'high_activity'
    WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
    WHEN invested_companies <20 THEN 'low_activity'
END 
FROM fund;

9. Рассчёт округлённого до ближайшего целого числа среднего количества инвестиционных раундов, в которых фонд принимал участие. Вывед на экран категорий и среднего числа инвестиционных раундов. Отсортировка таблицы по возрастанию среднего.

In [ ]:
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));

10. Анализ стран в который находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой странырассчитаное минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исклюеные страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгженоте десять самых активных стран-инвесторовсортировкате таблыцу по среднему количеству компаний от большего к меньшемуДобавленате сортироаку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
    MIN(invested_companies), 
    MAX(invested_companies), 
    AVG(invested_companies)
FROM fund
WHERE founded_at BETWEEN '2010-01-01' AND '2012-12-31'
GROUP BY country_code
HAVING MIN(invested_companies) != 0 
ORDER BY AVG(invested_companies) DESC, country_code 
LIMIT 10;

11. Имя и фамилию всех сотрудников стартапов. Добавлено поле с названием учебного заведения, которое окончил сотрудник.

In [ ]:
SELECT 
   first_name,
   last_name,
   instituition
FROM people AS p
LEFT JOIN education AS e ON p.id=e.person_id;

12. Количество учебных заведений, которые окончили её сотрудники. Выведено название компании и число уникальных названий учебных заведений. Составлено топ-5 компаний по количеству университетов.

In [ ]:
SELECT
    c.name,
    COUNT(DISTINCT instituition)
FROM company AS c
JOIN people AS p ON c.id=p.company_id
JOIN education AS e ON p.id=e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT instituition) DESC
LIMIT 5;

13. Список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT
    DISTINCT(c.name)
FROM company AS c
LEFT JOIN funding_round AS fr ON c.id=fr.company_id
WHERE status = 'closed' 
    AND is_first_round = 1
    AND is_last_round = 1

14. Список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT 
    DISTINCT(p.id)
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
JOIN people AS p ON c.id=p.company_id
WHERE status = 'closed' 
    AND is_first_round = 1
    AND is_last_round = 1;

15. Составление таблицы, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT 
    DISTINCT (p.id),
    e.instituition
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
JOIN people AS p ON c.id=p.company_id
JOIN education AS e ON p.id=e.person_id
WHERE status = 'closed' 
    AND is_first_round = 1
    AND is_last_round = 1;

16. Количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учтено, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT 
    p.id,
    COUNT(e.instituition) AS total_instituition
FROM people AS p    
JOIN education AS e ON p.id=e.person_id    
WHERE company_id IN
(SELECT c.id AS company_id
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE status = 'closed' 
    AND is_first_round = 1
    AND is_last_round = 1)
    GROUP BY p.id ;

17. Дополнен предыдущий запрос и выведено среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Выведена только одна запись.

In [ ]:
SELECT AVG(total_instituition)
FROM
(SELECT 
    p.id,
    COUNT(e.instituition) AS total_instituition
FROM people AS p    
JOIN education AS e ON p.id=e.person_id    
WHERE company_id IN
(SELECT c.id AS company_id
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE status = 'closed' 
    AND is_first_round = 1
    AND is_last_round = 1)
    GROUP BY p.id) AS a;

18. Среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

In [ ]:
SELECT AVG(total_instituition)
FROM

(SELECT 
    p.id,
    COUNT(e.instituition) AS total_instituition
FROM people AS p    
JOIN education AS e ON p.id=e.person_id    
WHERE company_id IN
 
(SELECT c.id AS company_id
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.name LIKE 'Socialnet')
 
GROUP BY p.id) AS a;

19. Составление таблицы из полей:
- `name_of_fund` — название фонда;
- `name_of_company` — название компании;
- `amount` — сумма инвестиций, которую привлекла компания в раунде.
В таблицу вошли данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT
    f.name AS name_of_fund, 
    c.name AS name_of_company,
    fr.raised_amount AS amount
FROM investment AS i
JOIN company AS c ON i.company_id=c.id
JOIN funding_round AS fr ON i.funding_round_id=fr.id
JOIN fund AS f ON i.fund_id=f.id
WHERE c.milestones > 6
    AND EXTRACT(YEAR FROM CAST (fr.funded_at AS TIMESTAMP)) BETWEEN 2012 AND 2013; 

20. Выгрузка таблицы со следующими полями:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитываются те сделки, в которых сумма покупки равна нулю. Сортировка таблицы по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Вывелены первые десять строк  записи.

In [ ]:
SELECT
    cb.name AS buyer,
    price_amount,
    cs.name AS seller,
    cs.funding_total,
    ROUND(price_amount / cs.funding_total)
FROM acquisition AS a
JOIN company AS cb ON cb.id=a.acquiring_company_id
JOIN company AS cs ON cs.id=a.acquired_company_id
WHERE price_amount !=0
    AND cs.funding_total !=0
ORDER BY price_amount DESC, seller
LIMIT 10;

21. Выгрузка таблицы, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверено, что сумма инвестиций не равна нулю. Выведен также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT
    c.name,
    EXTRACT (MONTH FROM CAST(funded_at AS timestamp))

FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.category_code = 'social'
    AND EXTRACT (YEAR FROM CAST(fr.funded_at AS DATE)) BETWEEN 2010 AND 2013
    AND fr.raised_amount != 0;

22. Отобраны данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппированы данные по номеру месяца и получена таблица, в которой следующие поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.яце.

In [ ]:
WITH
fu AS
(SELECT
    EXTRACT (MONTH FROM CAST(fr.funded_at AS DATE)) AS f_month,
    COUNT(DISTINCT(f.name)) AS f_id
 FROM fund AS f
 LEFT JOIN investment AS i ON f.id=i.fund_id
 LEFT JOIN funding_round AS fr ON i.funding_round_id=fr.id
WHERE EXTRACT (YEAR FROM CAST(fr.funded_at AS DATE)) BETWEEN 2010 AND 2013
    AND f.country_code = 'USA'
GROUP BY f_month),    

co AS
(SELECT
   EXTRACT (MONTH FROM CAST(acquired_at AS DATE)) AS f_month,
    COUNT(acquired_company_id) AS count_c,
    SUM(price_amount) AS total_sum
FROM acquisition 
WHERE EXTRACT (YEAR FROM CAST(acquired_at AS DATE)) BETWEEN 2010 AND 2013
GROUP BY f_month) 

SELECT
    f.f_month,
    f.f_id,
    c.count_c,
    c.total_sum
FROM fu AS f 
LEFT JOIN co AS c ON f.f_month=c.f_month

23. Составлена сводная таблица и выведена средняя сумма инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год в отдельном поле. Сортировка таблицы по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH

inv_2011 AS
(SELECT country_code,
    AVG(funding_total) AS sum_2011
 FROM company
 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2011
 GROUP BY country_code),
 
inv_2012 AS
(SELECT country_code,
    AVG(funding_total) AS sum_2012
 FROM company
 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2012
 GROUP BY country_code),
      
inv_2013 AS
(SELECT country_code,
    AVG(funding_total) AS sum_2013
 FROM company
 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2013
 GROUP BY country_code)


SELECT inv_2011.country_code,
           inv_2011.sum_2011,
           inv_2012.sum_2012,
           inv_2013.sum_2013
FROM inv_2011
INNER JOIN inv_2012 ON inv_2011.country_code=inv_2012.country_code
INNER JOIN inv_2013 ON inv_2011.country_code=inv_2013.country_code
ORDER BY sum_2011 DESC;